# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
from my_secrets3 import IEX_CLOUD_API_TOKEN
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
491,YUM
492,ZBH
493,ZBRA
494,ZION


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [4]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
#https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}
#https://cloud.iexapis.com/stable/tops?token={IEX_CLOUD_API_TOKEN}&symbols={symbol}
data = requests.get(api_url).json()
data

{'avgTotalVolume': 54144630,
 'calculationPrice': 'close',
 'change': 0.96,
 'changePercent': 0.00502,
 'close': None,
 'closeSource': 'official',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': None,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 169,
 'iexBidSize': 102,
 'iexClose': 192.75,
 'iexCloseTime': 1717419693568,
 'iexLastUpdated': 1717419693568,
 'iexMarketPercent': 0.00045254011820768857,
 'iexOpen': 192.75,
 'iexOpenTime': 1717419693568,
 'iexRealtimePrice': 192.75,
 'iexRealtimeSize': 5,
 'iexVolume': 215,
 'lastTradeTime': 1717185599989,
 'latestPrice': 192.25,
 'latestSource': 'Close',
 'latestTime': 'May 31, 2024',
 'latestUpdate': 1717185600094,
 'latestVolume': None,
 'low': None,
 'lowSource': None,
 'lowTime': None,
 'marketCap': 294

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [5]:
price=data['latestPrice']
peratio=data['peRatio']
peratio

29.9

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_url=f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        new_series = pd.Series(
            [symbol, data[symbol]['quote']['latestPrice'], data[symbol]['quote']['peRatio'], 'N/A'],
            index=my_columns
        )

        # Use pd.concat to append the new series to the DataFrame
        final_dataframe = pd.concat([final_dataframe, new_series.to_frame().T], ignore_index=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,130.41,31.12,N/A
1,AAL,11.5,17.54,N/A
2,AAP,70.64,155.59,N/A
3,AAPL,192.25,29.9,N/A
4,ABBV,161.24,47.99,N/A
...,...,...,...,...
491,YUM,137.43,24.32,N/A
492,ZBH,115.15,24.92,N/A
493,ZBRA,312.34,61.85,N/A
494,ZION,43.19,10.82,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [8]:
final_dataframe.sort_values('Price-to-Earnings Ratio',ascending=False,inplace=True)
final_dataframe=final_dataframe[:50]
final_dataframe.reset_index(drop=True,inplace=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,KSU,293.59,279.61,N/A
1,AMD,166.9,244.08,N/A
2,GILD,64.27,166.59,N/A
3,VNO,24.52,157.28,N/A
4,AAP,70.64,155.59,N/A
5,MRK,125.54,138.57,N/A
6,WELL,103.67,126.66,N/A
7,IRM,80.69,122.69,N/A
8,LLY,820.34,120.82,N/A
9,DIS,103.91,112.36,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [10]:
portfolio_input()
portfolio_size

'10000000'

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [11]:
# Assuming portfolio_size is a string, convert it to a float
portfolio_size = float(portfolio_size)

# Calculate position size
position_size = portfolio_size / len(final_dataframe.index)

# Calculate the number of shares to buy for each stock in the dataframe
final_dataframe['Number of Shares to Buy'] = final_dataframe['Price'].apply(lambda price: math.floor(position_size / price))

final_dataframe

C:\Users\iamha\AppData\Local\Temp\ipykernel_22312\1791483037.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataframe['Number of Shares to Buy'] = final_dataframe['Price'].apply(lambda price: math.floor(position_size / price))


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,KSU,293.59,279.61,681
1,AMD,166.9,244.08,1198
2,GILD,64.27,166.59,3111
3,VNO,24.52,157.28,8156
4,AAP,70.64,155.59,2831
5,MRK,125.54,138.57,1593
6,WELL,103.67,126.66,1929
7,IRM,80.69,122.69,2478
8,LLY,820.34,120.82,243
9,DIS,103.91,112.36,1924


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [12]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/advanced-stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
# Price-to-earnings ratio

# Price-to-book ratio
data['priceToBook']
# Price-to-sales ratio
data['priceToSales']
# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
data['enterpriseValue']/data['EBITDA']
# Enterprise Value divided by Gross Profit (EV/GP)
data['enterpriseValue']/data['grossProfit']

17.16092376958716

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [13]:
rv_columns = ['Ticker', 
              'Price',
              'Number of Shares to Buy',
              'PE ratio',
              'PE ratio Percentile',
              'PB ratio',
              'PB ratio Percentile',
              'PS ratio',
              'PS ratio Percentile',
              'EV/EBITDA',
              'EV/EBITDA Percentile',
              'EV/GP',
              'EV/GP Percentile',
              'RV Score'
              ]

rv_dataframe = pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        try:
            evebita = data[symbol]['advanced-stats']['enterpriseValue'] / data[symbol]['advanced-stats']['EBITDA']
        except TypeError:
            evebita = pd.NA
        try:
            evgp = data[symbol]['advanced-stats']['enterpriseValue'] / data[symbol]['advanced-stats']['grossProfit']
        except TypeError:
            evgp = pd.NA
        
        new_series = pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                evebita,
                'N/A',
                evgp,
                'N/A',
                'N/A'
            ],
            index=rv_columns
        )

        # use pd.concat to append the new series to the DataFrame
        rv_dataframe = pd.concat([rv_dataframe, new_series.to_frame().T], ignore_index=True)

rv_dataframe


,Ticker,Price,Number of Shares to Buy,PE ratio,PE ratio Percentile,PB ratio,PB ratio Percentile,PS ratio,PS ratio Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,130.41,N/A,31.12,N/A,6.18,N/A,5.67,N/A,24.7931,N/A,11.460893,N/A,N/A
1,AAL,11.5,N/A,17.54,N/A,-1.37,N/A,0.1419,N/A,5.688171,N/A,0.593862,N/A,N/A
2,AAP,70.64,N/A,155.59,N/A,1.64,N/A,0.3735,N/A,13.240584,N/A,1.23785,N/A,N/A
3,AAPL,192.25,N/A,29.9,N/A,39.73,N/A,7.72,N/A,23.030474,N/A,17.160924,N/A,N/A
4,ABBV,161.24,N/A,47.99,N/A,35.56,N/A,5.23,N/A,15.445954,N/A,10.054915,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,137.43,N/A,24.32,N/A,-4.99,N/A,5.51,N/A,20.122891,N/A,9.364913,N/A,N/A
492,ZBH,115.15,N/A,24.92,N/A,1.88,N/A,3.18,N/A,14.249314,N/A,5.430704,N/A,N/A
493,ZBRA,312.34,N/A,61.85,N/A,5.07,N/A,3.69,N/A,28.878744,N/A,8.925377,N/A,N/A
494,ZION,43.19,N/A,10.82,N/A,1.18,N/A,1.34,N/A,6.762681,N/A,1.371148,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [14]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,PE ratio,PE ratio Percentile,PB ratio,PB ratio Percentile,PS ratio,PS ratio Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
5,ABMD,381.02,N/A,65.69,N/A,None,N/A,None,N/A,<NA>,N/A,<NA>,N/A,N/A
27,ALXN,182.5,N/A,59.45,N/A,None,N/A,None,N/A,<NA>,N/A,<NA>,N/A,N/A
38,AON,281.64,N/A,None,N/A,None,N/A,None,N/A,<NA>,N/A,<NA>,N/A,N/A
46,ATVI,94.42,N/A,34.59,N/A,None,N/A,None,N/A,<NA>,N/A,<NA>,N/A,N/A
86,CERN,94.92,N/A,47.94,N/A,None,N/A,None,N/A,<NA>,N/A,<NA>,N/A,N/A
116,CTL,11,N/A,9.73,N/A,None,N/A,None,N/A,<NA>,N/A,<NA>,N/A,N/A
119,CTXS,103.9,N/A,39.96,N/A,None,N/A,None,N/A,<NA>,N/A,<NA>,N/A,N/A
133,DISCK,24.42,N/A,None,N/A,None,N/A,None,N/A,<NA>,N/A,<NA>,N/A,N/A
134,DISH,5.77,N/A,2.9,N/A,None,N/A,None,N/A,<NA>,N/A,<NA>,N/A,N/A
162,ETFC,49.26,N/A,14.27,N/A,None,N/A,None,N/A,<NA>,N/A,<NA>,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [15]:
for column in ['PE ratio','PB ratio','PS ratio','EV/EBITDA','EV/GP',]:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(),inplace=True)

C:\Users\iamha\AppData\Local\Temp\ipykernel_22312\3597462842.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  rv_dataframe[column].fillna(rv_dataframe[column].mean(),inplace=True)
C:\Users\iamha\AppData\Local\Temp\ipykernel_22312\3597462842.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rv_dataframe[col

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [16]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,PE ratio,PE ratio Percentile,PB ratio,PB ratio Percentile,PS ratio,PS ratio Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [17]:
metrics={
        'PE ratio':'PE ratio Percentile',
        'PB ratio':'PB ratio Percentile',
        'PS ratio':'PS ratio Percentile',
        'EV/EBITDA':'EV/EBITDA Percentile',
        'EV/GP':'EV/GP Percentile'
        }
for metric in metrics:
    for row in rv_dataframe.index:
        rv_dataframe.loc[row,metrics[metric]]=stats.percentileofscore(rv_dataframe[metric],rv_dataframe.loc[row,metric])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,PE ratio,PE ratio Percentile,PB ratio,PB ratio Percentile,PS ratio,PS ratio Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,130.41,N/A,31.12,70.967742,6.18,71.370968,5.6700,79.83871,24.793100,87.5,11.460893,80.645161,N/A
1,AAL,11.5,N/A,17.54,36.491935,-1.37,6.25,0.1419,0.806452,5.688171,6.048387,0.593862,2.016129,N/A
2,AAP,70.64,N/A,155.59,99.193548,1.64,23.58871,0.3735,4.435484,13.240584,44.153226,1.237850,7.258065,N/A
3,AAPL,192.25,N/A,29.90,68.75,39.73,97.580645,7.7200,87.5,23.030474,85.080645,17.160924,93.145161,N/A
4,ABBV,161.24,N/A,47.99,88.306452,35.56,97.379032,5.2300,77.419355,15.445954,57.862903,10.054915,72.580645,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,137.43,N/A,24.32,56.451613,-4.99,5.645161,5.5100,79.233871,20.122891,78.629032,9.364913,69.153226,N/A
492,ZBH,115.15,N/A,24.92,57.459677,1.88,29.737903,3.1800,56.350806,14.249314,48.58871,5.430704,39.919355,N/A
493,ZBRA,312.34,N/A,61.85,93.346774,5.07,66.330645,3.6900,61.693548,28.878744,92.33871,8.925377,66.935484,N/A
494,ZION,43.19,N/A,10.82,15.927419,1.18,13.608871,1.3400,24.395161,6.762681,9.475806,1.371148,9.879032,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [18]:
from statistics import mean
for row in rv_dataframe.index:
    values=[]
    for metric in metrics:
        values.append(rv_dataframe.loc[row,metrics[metric]])
    rv_dataframe.loc[row,'RV Score']=mean(values)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,PE ratio,PE ratio Percentile,PB ratio,PB ratio Percentile,PS ratio,PS ratio Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,130.41,N/A,31.12,70.967742,6.18,71.370968,5.6700,79.83871,24.793100,87.5,11.460893,80.645161,78.064516
1,AAL,11.5,N/A,17.54,36.491935,-1.37,6.25,0.1419,0.806452,5.688171,6.048387,0.593862,2.016129,10.322581
2,AAP,70.64,N/A,155.59,99.193548,1.64,23.58871,0.3735,4.435484,13.240584,44.153226,1.237850,7.258065,35.725806
3,AAPL,192.25,N/A,29.90,68.75,39.73,97.580645,7.7200,87.5,23.030474,85.080645,17.160924,93.145161,86.41129
4,ABBV,161.24,N/A,47.99,88.306452,35.56,97.379032,5.2300,77.419355,15.445954,57.862903,10.054915,72.580645,78.709677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,137.43,N/A,24.32,56.451613,-4.99,5.645161,5.5100,79.233871,20.122891,78.629032,9.364913,69.153226,57.822581
492,ZBH,115.15,N/A,24.92,57.459677,1.88,29.737903,3.1800,56.350806,14.249314,48.58871,5.430704,39.919355,46.41129
493,ZBRA,312.34,N/A,61.85,93.346774,5.07,66.330645,3.6900,61.693548,28.878744,92.33871,8.925377,66.935484,76.129032
494,ZION,43.19,N/A,10.82,15.927419,1.18,13.608871,1.3400,24.395161,6.762681,9.475806,1.371148,9.879032,14.657258


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [19]:
rv_dataframe.sort_values('RV Score',ascending=False,inplace=True)
rv_dataframe=rv_dataframe[:50]
rv_dataframe.reset_index(drop=True,inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,PE ratio,PE ratio Percentile,PB ratio,PB ratio Percentile,PS ratio,PS ratio Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,LLY,820.34,N/A,120.82,98.387097,60.85,98.790323,21.70,99.798387,62.214356,98.790323,27.888687,99.596774,99.072581
1,NVDA,1096.33,N/A,64.19,94.153226,54.88,98.185484,33.81,100.0,54.292112,97.782258,44.543595,100.0,98.024194
2,CDNS,286.31,N/A,74.56,96.169355,21.84,95.362903,19.11,99.395161,64.128625,98.991935,21.295825,97.983871,97.580645
3,DXCM,118.77,N/A,79.18,96.975806,21.02,94.959677,12.42,96.572581,71.857464,99.193548,19.608705,96.975806,96.935484
4,ISRG,402.12,N/A,72.58,95.967742,10.22,87.903226,19.49,99.596774,60.267540,98.58871,28.349383,99.798387,96.370968
5,NOW,656.93,N/A,70.34,95.564516,16.61,93.548387,14.21,98.185484,86.443701,99.596774,17.531592,93.75,96.129032
6,SNPS,560.8,N/A,61.16,93.145161,12.01,90.322581,13.88,97.983871,57.117329,98.387097,17.126908,92.943548,94.556452
7,VRSK,252.78,N/A,47.16,87.298387,127.80,99.193548,13.19,97.782258,26.798435,89.314516,20.907919,97.782258,94.274194
8,IDXX,496.95,N/A,48.15,88.508065,26.16,96.572581,11.02,95.16129,33.753608,95.362903,18.566444,95.362903,94.193548
9,AVGO,1328.55,N/A,49.85,89.112903,8.76,85.887097,15.84,98.790323,34.554180,95.564516,23.557872,99.395161,93.75


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [20]:
portfolio_input()
portfolio_size

'10000000'

In [21]:
# Assuming portfolio_size is a string, convert it to a float
portfolio_size = float(portfolio_size)

# Calculate position size
position_size = portfolio_size / len(rv_dataframe.index)

# Calculate the number of shares to buy for each stock in the dataframe
rv_dataframe['Number of Shares to Buy'] = rv_dataframe['Price'].apply(lambda price: math.floor(position_size / price))

rv_dataframe

C:\Users\iamha\AppData\Local\Temp\ipykernel_22312\2943139090.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_dataframe['Number of Shares to Buy'] = rv_dataframe['Price'].apply(lambda price: math.floor(position_size / price))


,Ticker,Price,Number of Shares to Buy,PE ratio,PE ratio Percentile,PB ratio,PB ratio Percentile,PS ratio,PS ratio Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,LLY,820.34,243,120.82,98.387097,60.85,98.790323,21.70,99.798387,62.214356,98.790323,27.888687,99.596774,99.072581
1,NVDA,1096.33,182,64.19,94.153226,54.88,98.185484,33.81,100.0,54.292112,97.782258,44.543595,100.0,98.024194
2,CDNS,286.31,698,74.56,96.169355,21.84,95.362903,19.11,99.395161,64.128625,98.991935,21.295825,97.983871,97.580645
3,DXCM,118.77,1683,79.18,96.975806,21.02,94.959677,12.42,96.572581,71.857464,99.193548,19.608705,96.975806,96.935484
4,ISRG,402.12,497,72.58,95.967742,10.22,87.903226,19.49,99.596774,60.267540,98.58871,28.349383,99.798387,96.370968
5,NOW,656.93,304,70.34,95.564516,16.61,93.548387,14.21,98.185484,86.443701,99.596774,17.531592,93.75,96.129032
6,SNPS,560.8,356,61.16,93.145161,12.01,90.322581,13.88,97.983871,57.117329,98.387097,17.126908,92.943548,94.556452
7,VRSK,252.78,791,47.16,87.298387,127.80,99.193548,13.19,97.782258,26.798435,89.314516,20.907919,97.782258,94.274194
8,IDXX,496.95,402,48.15,88.508065,26.16,96.572581,11.02,95.16129,33.753608,95.362903,18.566444,95.362903,94.193548
9,AVGO,1328.55,150,49.85,89.112903,8.76,85.887097,15.84,98.790323,34.554180,95.564516,23.557872,99.395161,93.75


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [22]:
# Define the Excel writer and the sheet name
writer = pd.ExcelWriter('recommended_trades3.xlsx', engine='xlsxwriter')

# Write the DataFrame to an Excel sheet
rv_dataframe.to_excel(writer, sheet_name='Recommended Trades3', index=False)

# Get the workbook and worksheet objects
workbook = writer.book
worksheet = writer.sheets['Recommended Trades3']

# Define formats
background_color = '#0a0a24'
font_color = '#ffffff'

string_format = workbook.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = workbook.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = workbook.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
percent_format = workbook.add_format(
    {
        'num_format': '0.00%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

# Apply formats to columns
for col_num, col_name in enumerate(rv_columns):
    if col_name == 'Price':
        worksheet.set_column(col_num, col_num, 18, dollar_format)
    elif col_name == 'Number of Shares to Buy':
        worksheet.set_column(col_num, col_num, 18, integer_format)
    elif col_name == 'Ticker':
        worksheet.set_column(col_num, col_num, 18, string_format)
    else:
        worksheet.set_column(col_num, col_num, 18, percent_format)

# Close the Excel writer
writer.close()